In [ ]:
# 获取token
import requests
import os

login_url = "https://admin.summerfarm.net/authentication/auth/username/login"
login_data = {
    "username": "peng.tang@summerfarm.net",
    "password": os.getenv('XIANMU_ADMIN_PASSWORD'),
}

token = requests.post(login_url, data=login_data).json()

print(token)


headers = {
    'token': token.get("data").get("token"),
    'xm-rqid': 'create_fake_merchant_tp',
    'xm-uid': '2047',
    'Content-Type': 'application/json;charset=UTF-8',
}

print(headers)

In [ ]:
import requests

url = 'https://admin.summerfarm.net/large-area/larges'
large_area = requests.get(url, headers=headers).json()
large_area

In [ ]:
update_member_grade_json = {
    "areaName": "测试大区域",
    "storeNo": 1,
    "companyAccountId": 1,
    "companyName": "海南水木铭农业科技有限公司",
    "mapSection": [],
    "areaNo": 29316,
    "status": 1,
    "deliveryFee": 10,
    "expressFee": 11,
    "deliveryRule": '{"categoryList":[]}',
    "memberRule": '[{"grade":0,"threshold":0,"refundAmount":0,"outTimes":0},{"grade":1,"threshold":1000,"refundAmount":10,"outTimes":100},{"grade":2,"threshold":2000,"refundAmount":300,"outTimes":200},{"grade":3,"threshold":4000,"refundAmount":500,"outTimes":500}]',
    "freeDay": "",
    "notifyTitle": "",
    "notifyContent": "",
    "notifyRemarks": "",
    "changeFlag": False,
    "changeStatus": 0,
    "changeStoreNo": "",
    "changStartTime": "",
    "administrativeArea": "河南/洛阳市",
    "adminId": 12,
    "supportAddOrder": 1,
    "largeAreaNo": 1,
    "largeAreaName": "测试大区域",
    "largeName": "杭州大区",
    "payChannel": 0,
    "nextDeliveryDate": "",
    "parentNo": 1,
}

In [ ]:
import pandas as pd
new_grade_df=pd.read_csv("./新会员等级-分省份2024.csv")
new_grade_df.head(2)

In [ ]:
print(new_grade_df['省份'].unique())

In [ ]:
all_sub_area=[]

for large_area_obj in large_area.get('data'):
    print(large_area_obj)
    if int(large_area_obj.get('status'))==0:
        print(f"服务区关闭的:{large_area_obj}")
        continue
    if '测试' in large_area_obj.get('largeAreaName'):
        print(f"服务大区是测试区域:{large_area_obj}")
    for area in large_area_obj.get("areaList"):
        if int(area.get('status'))==0:
            print(f"服务小区是关闭的:{area}")
            continue
        if '测试' in area.get('areaName',''):
            print(f"服务小区是测试区域:{area}")
            continue
        area['largeName']=large_area_obj['largeAreaName']
        all_sub_area.append(area)
    print(all_sub_area[len(all_sub_area)-1])

In [ ]:
all_sub_area_df=pd.DataFrame(all_sub_area)

In [ ]:
import json
new_grade_df.head(2)

new_grade_map={}

def create_member_rule(row):
    grade_json= [
        {"grade": 0, "threshold": row['交易门槛V0'], "refundAmount": row['极速售后额度V0'], "outTimes": row['超时加单次数']},
        {"grade": 1, "threshold": row['交易门槛V1'], "refundAmount": row['极速售后额度V1'], "outTimes": row['超时加单次数']},
        {"grade": 2, "threshold": row['交易门槛V2'], "refundAmount": row['极速售后额度V2'], "outTimes": row['超时加单次数']},
        {"grade": 3, "threshold": row['交易门槛V3'], "refundAmount": row['极速售后额度V3'], "outTimes": row['超时加单次数']}
    ]

    new_grade_map[f"{row['省份']}_{row['对应运营大区']}"]=json.dumps(grade_json)

    return grade_json

new_grade_df['memberRule'] = new_grade_df.apply(create_member_rule, axis=1)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
new_grade_df[['省份','对应运营大区','memberRule']]

In [ ]:
new_grade_map

In [ ]:
all_sub_area[0]
not_found_areas = []
all_sql=""
for area in all_sub_area:

    area_of_this = f"{area['areaName']}:{area['areaNo']}"
    province = area["administrativeArea"].split("/")[0]
    if len(province) > 3:
        province = province[:2]
    large_name = area["largeName"]
    # print(province, large_name)
    new_grade_json = new_grade_map.get(f"{province}_{large_name}", "呵呵获取不到新规则")
    if "呵呵" in new_grade_json:
        # print(f"本运营服务区找不到新会员规则: {area}")
        not_found_areas.append(area)
        continue

    new_grade_json = json.loads(new_grade_json)
    old_rule = area["memberRule"]
    old_rule_json = None
    if old_rule is not None and len(old_rule) > 10:
        old_rule_json = json.loads(old_rule)
        # print(old_rule_json)
        for rule in new_grade_json:
            # print(f"rule:{rule}")
            for old in old_rule_json:
                # print(f"old:{old}")
                if rule["grade"] == old["grade"]:
                    rule["outTimes"] = old["outTimes"]
                    break
    print(
        f"======>\n运营服务区：{area_of_this},{province},{large_name}, \n旧规则：{old_rule_json}, \n新规则：{new_grade_json}\n\n"
    )
    all_sql=f"""{all_sql}update `area` set `member_rule`='{json.dumps(new_grade_json)}',update_time=now() 
where area_no = {area['areaNo']} and area_name='{area['areaName']}' and large_area_no={area['largeAreaNo']};\n\n"""

not_found_areas_df = pd.DataFrame(not_found_areas)
not_found_areas_df[["administrativeArea", "areaName", "areaNo", "largeName"]]

print("以下是找不到新规则的运营服务区！")
for index, row in not_found_areas_df[["administrativeArea", "areaName", "areaNo", "largeName"]].iterrows():
    print(f"找不到新规则:{row.to_dict()}")

In [ ]:
print("SQL：{}")
print(all_sql)